# Chapter 4 Exercise 2
- ## Write a query that returns all orders placed by the customer(s) who placed the highest number of orders
- ## Table involved: Sales.Order

### Desired Output:
| custid | orderid | orderdate  | empid |
|--------|---------|------------|-------|
| 71     | 10324   | 2014-10-08 | 9     |
| 71     | 10393   | 2014-12-25 | 1     |
| 71     | 10398   | 2014-12-30 | 2     |
| 71     | 10440   | 2015-02-10 | 4     |
| 71     | 10452   | 2015-02-20 | 8     |
| 71     | 10510   | 2015-04-18 | 6     |
| 71     | 10555   | 2015-06-02 | 6     |
| 71     | 10603   | 2015-07-18 | 8     |
| 71     | 10607   | 2015-07-22 | 5     |
| 71     | 10612   | 2015-07-28 | 1     |
| 71     | 10627   | 2015-08-11 | 8     |
| 71     | 10657   | 2015-09-04 | 2     |
| 71     | 10678   | 2015-09-23 | 7     |
| 71     | 10700   | 2015-10-10 | 3     |
| 71     | 10711   | 2015-10-21 | 5     |
| 71     | 10713   | 2015-10-22 | 1     |
| 71     | 10714   | 2015-10-22 | 5     |
| 71     | 10722   | 2015-10-29 | 8     |
| 71     | 10748   | 2015-11-20 | 3     |
| 71     | 10757   | 2015-11-27 | 6     |
| 71     | 10815   | 2016-01-05 | 2     |
| 71     | 10847   | 2016-01-22 | 4     |
| 71     | 10882   | 2016-02-11 | 4     |
| 71     | 10894   | 2016-02-18 | 1     |
| 71     | 10941   | 2016-03-11 | 7     |
| 71     | 10983   | 2016-03-27 | 2     |
| 71     | 10984   | 2016-03-30 | 1     |
| 71     | 11002   | 2016-04-06 | 4     |
| 71     | 11030   | 2016-04-17 | 7     |
| 71     | 11031   | 2016-04-17 | 6     |
| 71     | 11064   | 2016-05-01 | 1     |

(31 row(s) affected)

In [ ]:
USE Northwinds2022TSQLV7

select CustomerId, OrderId, OrderDate, EmployeeId
from Sales.[order]
where customerid in 
(SELECT TOP (1) WITH TIES           -- Subquery that retrieves the customer with the most orders
    O.CustomerId
FROM Sales.[Order] AS O
GROUP BY O.CustomerId
ORDER BY COUNT(*) DESC)

# Chapter 4 Exercise 9
## - Explain the difference between IN and EXISTS.

### Explanation: 
The IN operator is used to find multiple values from a list or set matching one-to-many relationships, such as in a book made by multiple authors. The EXISTS operator, on the other hand, detects the presence or absence of a row, regardless of whether it returns a null value or not ie it will consider both null and non-null values of each row. EXISTS requires a subquery to function and can be used within correlated subqueries while the IN operator does not require subqueries but cannot be used within correlated subqueries. The EXISTS operator is generally more efficient as it only detects the existence of a row and stops evaluating once it finds at least one match returning faster results, unlike the IN operator which continues evaluating and comparing all values in the list.

# Chapter 5 Exercise 4
- ## Write a solution using a recursive CTE that returns the management chain leading to Patricia Doyle (employee ID 9):
    
- ## Table involved: HumanResources.Employee
    
### Desired Output:
| empid | mgrid | firstname | lastname |
| --- | --- | --- | --- |
| 9 | 5 | Patricia | Doyle |
| 5 | 2 | Sven | Mortensen |
| 2 | 1 | Don | Funk |
| 1 | NULL | Sara | Davis |
(4 row(s) affected)

In [ ]:
USE Northwinds2022TSQLV7;

WITH
    EmpCTE
    AS
    (
                    SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
            FROM HumanResources.Employee
            WHERE EmployeeId = 9
        UNION ALL
            SELECT E.EmployeeId, E.EmployeeManagerId, E.EmployeeFirstName, E.EmployeeLastName
            FROM EmpCTE AS C
                INNER JOIN HumanResources.Employee AS E
                ON C.EmployeeManagerId = E.EmployeeId
    )

SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
FROM EmpCTE;

# Proposition 1

Retrieve orders with total values greater than the average order line value.

# Functional Specification

## Query Name
Orders with Total Values Above Average Order Line Value

## Description
This query retrieves order details from the `Sales.Orders` table along with their corresponding total values from the `Sales.OrderLines` table. It filters the results to include only those orders whose total value (calculated as `Quantity * UnitPrice`) exceeds the average order line value across all orders. Additionally, it provides the average value for reference.

## Inputs
- **OrderID**: The unique identifier for each order.
- **CustomerID**: The unique identifier for each customer.
- **OrderDate**: The date when the order was placed.
- **Quantity**: The quantity of items ordered.
- **UnitPrice**: The price per unit of the ordered items.

## Outputs
- **OrderID**: The unique identifier for the order.
- **CustomerID**: The customer associated with the order.
- **OrderDate**: The date the order was placed.
- **TotalValue**: The total value of the order line, calculated as `Quantity * UnitPrice`.
- **AverageValue**: The average value of all order lines, rounded to two decimal places.

## Steps
1. Join the `Sales.Orders` table and `Sales.OrderLines` table on `OrderID`.
2. Calculate the total value for each order line as `Quantity * UnitPrice`.
3. Compute the average order line value using a subquery.
4. Filter out order lines where the total value is less than or equal to the average value.

## Assumptions
- There are no NULL values in the `Quantity` or `UnitPrice` fields.
- All monetary values are expressed in the same currency.

## Example Output

| OrderID | CustomerID | OrderDate   | TotalValue | AverageValue |
|---------|------------|-------------|------------|--------------|
| 1001    | C001       | 2024-01-12  | 500.00     | 300.45       |
| 1002    | C002       | 2024-02-15  | 450.00     | 300.45       |



In [ ]:
USE WideWorldImporters

SELECT O.OrderID, CustomerID, OrderDate, Quantity* UnitPrice as TotalValue, (SELECT round(AVG(Quantity*UnitPrice), 2)
    FROM Sales.OrderLines) as AverageValue
FROM Sales.Orders as O
    JOIN Sales.OrderLines as OL on Ol.OrderID = O.OrderID
WHERE  Quantity* UnitPrice > (
    SELECT AVG(Quantity*UnitPrice)
FROM Sales.OrderLines
);


# Proposition 2

Retrieve stock items with prices below the average price and high sales volume.

# Functional Specification

## Query Name
Stock Items Below Average Price with High Sales Volume

## Description
This query retrieves stock item details from the `Warehouse.StockItems` table. It filters the results to include only those stock items whose unit price is below the average unit price across all stock items and have been ordered more than 1000 times. The query utilizes subqueries to determine the average price and to filter stock items based on sales volume.

## Inputs
- **StockItemID**: The unique identifier for each stock item.
- **StockItemName**: The name of the stock item.
- **UnitPrice**: The price per unit of the stock item.

## Outputs
- **StockItemID**: The unique identifier for the stock item.
- **StockItemName**: The name of the stock item.
- **UnitPrice**: The unit price of the stock item.

## Steps
1. Select stock item details (`StockItemID`, `StockItemName`, `UnitPrice`) from the `Warehouse.StockItems` table.
2. Filter stock items where the unit price is less than the average unit price computed from the same table.
3. Further filter the results to include only those stock items that have been ordered more than 1000 times, determined through a subquery on the `Sales.OrderLines` table.
4. Group the results by `StockItemID`, `StockItemName`, and `UnitPrice`.

## Assumptions
- The data is available in the `Warehouse.StockItems` and `Sales.OrderLines` tables.
- The `StockItemID` is unique across the `Warehouse.StockItems` table.

## Example Output

| StockItemID | StockItemName   | UnitPrice |
|--------------|------------------|-----------|
| 101          | Widget A         | 9.99      |
| 202          | Widget B         | 14.50     |
| 303          | Widget C         | 7.25      |



In [ ]:
USE WideWorldImporters

SELECT StockItemID, StockItemName, UnitPrice
FROM Warehouse.StockItems
WHERE UnitPrice < (
    SELECT AVG(UnitPrice)
    FROM Warehouse.StockItems
)
    AND StockItemID IN (
    SELECT StockItemID
    FROM Sales.OrderLines
    GROUP BY StockItemID
    HAVING COUNT(OrderID) > 1000
)
group by StockItemID, StockItemName, UnitPrice

# Proposition 3

Retrieve suppliers of stock items with prices above the average price.

# Functional Specification

## Query Name
Suppliers of Stock Items Above Average Price

## Description
This query retrieves supplier details from the `Purchasing.Suppliers` table for those suppliers who provide stock items priced above the average unit price across all stock items in the `Warehouse.StockItems` table. The query utilizes subqueries to determine the average price and to filter suppliers based on stock item pricing.

## Inputs
- **SupplierID**: The unique identifier for each supplier.
- **SupplierName**: The name of the supplier.
- **UnitPrice**: The price per unit of the stock item.

## Outputs
- **SupplierID**: The unique identifier for the supplier.
- **SupplierName**: The name of the supplier.

## Steps
1. Select supplier details (`SupplierID`, `SupplierName`) from the `Purchasing.Suppliers` table.
2. Filter suppliers where the `SupplierID` is present in the subquery that selects suppliers from the `Warehouse.StockItems` table.
3. The subquery checks for stock items where the unit price is greater than the average unit price calculated from the same table.

## Assumptions
- The data is available in the `Purchasing.Suppliers` and `Warehouse.StockItems` tables.
- The `SupplierID` is unique across the `Purchasing.Suppliers` table.

## Example Output

| SupplierID | SupplierName      |
|------------|-------------------|
| 1          | Acme Corp         |
| 2          | Global Supplies    |
| 3          | Widgetry Inc.     |


In [ ]:
USE WideWorldImporters

SELECT SupplierID, SupplierName
FROM Purchasing.Suppliers
WHERE SupplierID IN (
    SELECT SupplierID
    FROM Warehouse.StockItems
    WHERE UnitPrice > (
        SELECT AVG(UnitPrice)
        FROM Warehouse.StockItems
    )
);

# Proposition 4

Retrieve orders containing the stock item with the lowest price.

# Functional Specification

## Query Name
Orders with the Cheapest Stock Item

## Description
This query retrieves order details from the `Sales.Orders` table for orders that include the stock item with the lowest unit price. It utilizes nested subqueries to identify the stock item with the minimum price and to filter orders containing that item.

## Inputs
- **OrderID**: The unique identifier for each order.
- **OrderDate**: The date when the order was placed.
- **CustomerID**: The unique identifier for each customer.

## Outputs
- **OrderID**: The unique identifier for the order.
- **OrderDate**: The date the order was placed.
- **CustomerID**: The customer associated with the order.

## Steps
1. Select order details (`OrderID`, `OrderDate`, `CustomerID`) from the `Sales.Orders` table.
2. Filter orders where the `OrderID` is present in a subquery that retrieves distinct `OrderID`s from the `Sales.OrderLines` table.
3. The inner subquery identifies the `StockItemID` corresponding to the stock item with the minimum unit price from the `Warehouse.StockItems` table.
4. The outer subquery checks for orders that contain this stock item.

## Assumptions
- The data is available in the `Sales.Orders`, `Sales.OrderLines`, and `Warehouse.StockItems` tables.
- The `OrderID` is unique across the `Sales.Orders` table.

## Example Output

| OrderID | OrderDate   | CustomerID |
|---------|-------------|------------|
| 1001    | 2024-01-12  | C001       |
| 1002    | 2024-02-15  | C002       |
| 1003    | 2024-03-20  | C003       |


In [ ]:
USE WideWorldImporters

SELECT OrderID, OrderDate, CustomerID
FROM Sales.Orders
WHERE OrderID IN (
    SELECT DISTINCT OrderID
FROM Sales.OrderLines
WHERE StockItemID = (
        SELECT StockItemID
FROM Warehouse.StockItems
WHERE UnitPrice = (
            SELECT MIN(UnitPrice)
FROM Warehouse.StockItems
        )
    )
);

# Proposition 5

Retrieve suppliers of stock items that have been ordered more than 1000 times.

# Functional Specification

## Query Name
Suppliers of High-Volume Stock Items

## Description
This query retrieves supplier details from the `Purchasing.Suppliers` table for those suppliers that provide stock items that have been ordered more than 1000 times. The query uses nested subqueries to identify high-volume stock items and their corresponding suppliers.

## Inputs
- **SupplierID**: The unique identifier for each supplier.
- **SupplierName**: The name of the supplier.
- **StockItemID**: The unique identifier for each stock item.
- **OrderID**: The unique identifier for each order.

## Outputs
- **SupplierID**: The unique identifier for the supplier.
- **SupplierName**: The name of the supplier.

## Steps
1. Select supplier details (`SupplierID`, `SupplierName`) from the `Purchasing.Suppliers` table.
2. Filter suppliers where the `SupplierID` is present in a subquery that selects `SupplierID` from the `Warehouse.StockItems` table.
3. The inner subquery checks for `StockItemID`s that have been ordered more than 1000 times, determined through a `GROUP BY` clause and a `HAVING` clause on the `Sales.OrderLines` table.

## Assumptions
- The data is available in the `Purchasing.Suppliers`, `Warehouse.StockItems`, and `Sales.OrderLines` tables.
- The `SupplierID` is unique across the `Purchasing.Suppliers` table.

## Example Output

| SupplierID | SupplierName      |
|------------|-------------------|
| 1          | Acme Corp         |
| 2          | Global Supplies    |
| 3          | Widgetry Inc.     |


In [ ]:
USE WideWorldImporters

SELECT SupplierID, SupplierName
FROM Purchasing.Suppliers
WHERE SupplierID IN (
    SELECT SupplierID
FROM Warehouse.StockItems
WHERE StockItemID IN (
        SELECT StockItemID
FROM Sales.OrderLines
GROUP BY StockItemID
HAVING COUNT(OrderID) > 1000
    )
);


# Proposition 6

Retrieve the top 5 customers by total sales in the year 2015.

# Functional Specification

## Query Name
Top 5 Customers by Total Sales in 2015

## Description
This query retrieves the top 5 customers based on total sales calculated from invoices in the year 2015. It uses a Common Table Expression (CTE) to aggregate sales data and then selects the top customers based on their total sales.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **CustomerName**: The name of the customer.
- **Quantity**: The quantity of items sold.
- **UnitPrice**: The price per unit of the items sold.
- **InvoiceDate**: The date when the invoice was issued.

## Outputs
- **CustomerID**: The unique identifier for the customer.
- **CustomerName**: The name of the customer.
- **TotalSales**: The total sales amount for the customer in the year 2015.

## Steps
1. Define a Common Table Expression (CTE) named `CustomerSalesCTE` to calculate total sales for each customer in 2015.
   - Join `Sales.Invoices`, `Sales.Customers`, and `Sales.OrderLines` to aggregate sales data.
   - Filter invoices where the year of `InvoiceDate` is 2015.
   - Group results by `CustomerID` and `CustomerName`, calculating total sales as the sum of `Quantity * UnitPrice`.
2. Select the top 5 customers from the CTE based on `TotalSales`, ordering the results in descending order.

## Assumptions
- The data is available in the `Sales.Invoices`, `Sales.Customers`, and `Sales.OrderLines` tables.
- The `CustomerID` is unique across the `Sales.Customers` table.
- There are no NULL values in the relevant fields.

## Example Output

| CustomerID | CustomerName      | TotalSales |
|------------|-------------------|------------|
| 101        | John Doe          | 15000.00   |
| 102        | Jane Smith        | 12000.00   |
| 103        | Acme Corp         | 11000.00   |
| 104        | Global Supplies    | 10500.00   |
| 105        | Widgetry Inc.     | 10000.00   |


In [ ]:
USE WideWorldImporters;

WITH
    CustomerSalesCTE
    AS
    (
        SELECT
            c.CustomerID,
            c.CustomerName,
            SUM(Quantity*UnitPrice) AS TotalSales
        FROM Sales.Invoices i
            INNER JOIN Sales.Customers c ON i.CustomerID = c.CustomerID
            join Sales.OrderLines ol on i.OrderID = ol.OrderID
        WHERE YEAR(i.InvoiceDate) = 2015
        GROUP BY c.CustomerID, c.CustomerName
    )
SELECT top 5
    CustomerID,
    CustomerName,
    TotalSales
FROM CustomerSalesCTE
ORDER BY TotalSales DESC


# Proposition 7

Retrieve product profit margins from stock items in descending order.

# Functional Specification

## Query Name
Product Profit Margins

## Description
This query calculates and retrieves the profit margins for products listed in the `Warehouse.StockItems` table. It uses a Common Table Expression (CTE) to compute the profit margin for each stock item based on its recommended retail price and unit price, considering applicable tax rates. The results are then sorted in descending order of profit margin.

## Inputs
- **StockItemID**: The unique identifier for each stock item.
- **StockItemName**: The name of the stock item.
- **RecommendedRetailPrice**: The suggested selling price of the stock item.
- **UnitPrice**: The cost price of the stock item.
- **TaxRate**: The applicable tax rate for the stock item.

## Outputs
- **StockItemID**: The unique identifier for the stock item.
- **StockItemName**: The name of the stock item.
- **ProfitMargin**: The calculated profit margin for each stock item.

## Steps
1. Define a Common Table Expression (CTE) named `ProductMarginCTE` to calculate profit margins for stock items.
   - Select necessary fields from the `Warehouse.StockItems` table: `StockItemID`, `StockItemName`, `RecommendedRetailPrice`, `UnitPrice`, and `TaxRate`.
   - Calculate the profit margin as:
     \[
     \text{ProfitMargin} = \text{RecommendedRetailPrice} - \left( \text{UnitPrice} \times \left(1 + \frac{\text{TaxRate}}{100}\right) \right)
     \]
2. Select `StockItemID`, `StockItemName`, and `ProfitMargin` from the CTE.
3. Order the results by `ProfitMargin` in descending order.

## Assumptions
- The data is available in the `Warehouse.StockItems` table.
- The `StockItemID` is unique across the `Warehouse.StockItems` table.
- The `TaxRate` and prices are valid numeric values and not NULL.

## Example Output

| StockItemID | StockItemName      | ProfitMargin |
|--------------|---------------------|--------------|
| 201          | Premium Widget      | 25.00        |
| 202          | Advanced Gadget     | 20.00        |
| 203          | Basic Tool          | 15.00        |
| 204          | Economy Widget      | 10.00        |
| 205          | Standard Gadget     | 5.00         |


In [ ]:
USE WideWorldImporters;

WITH
    ProductMarginCTE
    AS
    (
        SELECT
            w.StockItemID,
            w.StockItemName, RecommendedRetailPrice, UnitPrice, TaxRate,
            (w.RecommendedRetailPrice - w.UnitPrice * (1+(w.TaxRate/100))) AS ProfitMargin
        FROM Warehouse.StockItems w
    )
SELECT
    StockItemID,
    StockItemName,
    ProfitMargin
FROM ProductMarginCTE
ORDER BY ProfitMargin DESC



# Proposition 8

Retrieve the count of products supplied by each supplier, ordered by product count.

# Functional Specification

## Query Name
Supplier Product Count

## Description
This query retrieves the number of products supplied by each supplier from the `Purchasing.Suppliers` table. It uses a Common Table Expression (CTE) to count the stock items associated with each supplier and returns the results sorted by the count of products in descending order.

## Inputs
- **SupplierID**: The unique identifier for each supplier.
- **SupplierName**: The name of the supplier.
- **StockItemID**: The unique identifier for each stock item.

## Outputs
- **SupplierID**: The unique identifier for the supplier.
- **SupplierName**: The name of the supplier.
- **ProductCount**: The total number of products supplied by the supplier.

## Steps
1. Define a Common Table Expression (CTE) named `SupplierProductCount` to calculate the number of products supplied by each supplier.
   - Join the `Warehouse.StockItems` and `Purchasing.Suppliers` tables based on `SupplierID`.
   - Count the number of distinct `StockItemID`s for each supplier.
   - Group results by `SupplierID` and `SupplierName`.
2. Select `SupplierID`, `SupplierName`, and `ProductCount` from the CTE.
3. Order the results by `ProductCount` in descending order.

## Assumptions
- The data is available in the `Warehouse.StockItems` and `Purchasing.Suppliers` tables.
- The `SupplierID` is unique across the `Purchasing.Suppliers` table.
- Each supplier may supply multiple stock items.

## Example Output

| SupplierID | SupplierName      | ProductCount |
|------------|-------------------|--------------|
| 1          | Acme Corp         | 50           |
| 2          | Global Supplies    | 45           |
| 3          | Widgetry Inc.     | 30           |
| 4          | Premium Products   | 25           |
| 5          | Basic Supplies     | 10           |



In [ ]:
USE WideWorldImporters;

WITH SupplierProductCount AS (
    SELECT 
        s.SupplierID,
        s.SupplierName,
        COUNT(si.StockItemID) AS ProductCount
    FROM Warehouse.StockItems si
    INNER JOIN Purchasing.Suppliers s ON si.SupplierID = s.SupplierID
    GROUP BY s.SupplierID, s.SupplierName
)
SELECT 
    SupplierID,
    SupplierName,
    ProductCount
FROM SupplierProductCount
ORDER BY ProductCount DESC;


# Proposition 9

Retrieve products with total sales exceeding 100,000, ordered by total sales.

# Functional Specification

## Query Name
High-Value Product Sales

## Description
This query retrieves products from the `Warehouse.StockItems` table that have total sales exceeding 100,000. It uses a Common Table Expression (CTE) to calculate total sales for each product based on order lines and sorts the results in descending order of total sales.

## Inputs
- **StockItemID**: The unique identifier for each stock item.
- **StockItemName**: The name of the stock item.
- **Quantity**: The quantity of items sold in each order line.
- **UnitPrice**: The price per unit of the items sold.

## Outputs
- **StockItemID**: The unique identifier for the stock item.
- **StockItemName**: The name of the stock item.
- **TotalSales**: The total sales amount for each stock item.

## Steps
1. Define a Common Table Expression (CTE) named `ProductSales` to calculate total sales for each product.
   - Join the `Sales.OrderLines` table with the `Warehouse.StockItems` table using `StockItemID`.
   - Calculate total sales as the sum of `Quantity * UnitPrice` for each product.
   - Group results by `StockItemID` and `StockItemName`.
2. Select all fields from the `ProductSales` CTE.
3. Filter the results to include only those products with `TotalSales` greater than 100,000.
4. Order the results by `TotalSales` in descending order.

## Assumptions
- The data is available in the `Sales.OrderLines` and `Warehouse.StockItems` tables.
- Each stock item can have multiple entries in the `Sales.OrderLines` table.

## Example Output

| StockItemID | StockItemName      | TotalSales |
|--------------|---------------------|------------|
| 301          | Elite Widget        | 150000.00  |
| 302          | Premium Gadget      | 125000.00  |
| 303          | Advanced Tool       | 110000.00  |
| 304          | Standard Product    | 105000.00  |


In [ ]:
USE WideworldImporters;

WITH
    ProductSales
    AS
    (
        SELECT p.StockItemID, p.StockItemName, SUM(od.Quantity * od.UnitPrice) AS TotalSales
        FROM Sales.OrderLines od
            JOIN Warehouse.StockItems p ON od.StockItemID = p.StockItemID
        GROUP BY p.StockItemID, p.StockItemName
    )
SELECT *
FROM ProductSales
WHERE TotalSales > 100000
ORDER BY TotalSales DESC;


# Proposition 10

Retrieve orders where the expected delivery date is in the following month, along with the delivery time in days.

# Functional Specification

## Query Name
Order Delivery Time Analysis

## Description
This query retrieves order details, including the delivery time in days, for orders where the expected delivery date falls in the month immediately following the order date. It uses a Common Table Expression (CTE) to calculate the number of days between the order date and the expected delivery date.

## Inputs
- **OrderID**: The unique identifier for each order.
- **CustomerID**: The unique identifier for each customer.
- **OrderDate**: The date when the order was placed.
- **ExpectedDeliveryDate**: The date when the order is expected to be delivered.
- **DeliveryDays**: The calculated number of days between the order date and the expected delivery date.

## Outputs
- **OrderID**: The unique identifier for the order.
- **CustomerID**: The unique identifier for the customer associated with the order.
- **OrderDate**: The date the order was placed.
- **ExpectedDeliveryDate**: The expected delivery date of the order.
- **DeliveryDays**: The number of days between the order date and the expected delivery date.

## Steps
1. Define a Common Table Expression (CTE) named `OrderDeliveryTime` to calculate the delivery days for each order.
   - Select the necessary fields: `OrderID`, `CustomerID`, `OrderDate`, `ExpectedDeliveryDate`.
   - Calculate the delivery time in days using the `DATEDIFF` function.
2. Select `OrderID`, `CustomerID`, `OrderDate`, `ExpectedDeliveryDate`, and `DeliveryDays` from the CTE.
3. Filter the results to include only those orders where the month of `ExpectedDeliveryDate` is the month following the month of `OrderDate`.
4. Order the results by `DeliveryDays` in ascending order.

## Assumptions
- The data is available in the `Sales.Orders` table.
- Dates are valid and correctly formatted.
- The month calculation assumes that the year of both dates is the same or properly handles year transitions.

## Example Output

| OrderID | CustomerID | OrderDate  | ExpectedDeliveryDate | DeliveryDays |
|---------|------------|------------|----------------------|--------------|
| 501     | 1001       | 2024-01-15 | 2024-02-15           | 31           |
| 502     | 1002       | 2024-01-20 | 2024-02-05           | 16           |
| 503     | 1003       | 2024-01-30 | 2024-02-28           | 29           |


In [ ]:
USE WideWorldImporters;

WITH
    OrderDeliveryTime
    AS
    (
        SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate,
            DATEDIFF(DAY, OrderDate, ExpectedDeliveryDate) AS DeliveryDays
        FROM Sales.Orders
    )
SELECT OrderID, CustomerID, OrderDate, ExpectedDeliveryDate, DeliveryDays
FROM OrderDeliveryTime
WHERE MONTH(OrderDate) + 1 = MONTH(ExpectedDeliveryDate)
ORDER BY DeliveryDays 
